Name: Christian Hellum Bye

# Predicting a Pulsar Star

Based on previous work, we have trained a Random Forest Classifier (RFC) as our model. In this deliverable, we explore the relative importance of the features and see if we can further improve our model.

## Preprocessing the data

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split #to split the dataset

In [3]:
data = np.loadtxt('pulsar_stars.csv', delimiter=',', skiprows=1)

In [4]:
X = data[:, 0:8] #features
y = data[:, 8] #classes

In [5]:
#split the dataset into two parts, 80 % containing training and validation sets, 20 % to the test set
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.2)

#split the larger part of the dataset to two parts: 75 % (= 60 % of the total data) to training set, 25 % (= 20 % of the total)
#to the validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.25)

## Feature importance

In [6]:
#import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt

In [34]:
rfc = RandomForestClassifier(n_estimators = 45, min_samples_split = 5, max_features = 4, class_weight='balanced')
rfc.fit(X_train, y_train) #fits to training set
    
test_predict = rfc.predict(X_test) #make predictions
rfc_test_f1 = f1_score(y_test, test_predict) #f1-score for test data

print('Test data:', rfc_test_f1)

Test data: 0.880248833592535


In [281]:
import pickle
pickle.dump(svm_clf, open('SVM_weigths.sav', 'wb'))
pickle.dump(rfc, open('RFC_weigths.sav', 'wb'))

The confusion matrix for our model is:

In [282]:
confusion_test = confusion_matrix(y_test, test_predict)
print('Confusion matrix: \n')
print(confusion_test)

tn, fp, fn, tp = confusion_test.ravel()

print('\nTrue positives', tp)
print('True negatives', tn)
print('False positives', fp)
print('False negatives', fn)

Confusion matrix: 

[[3238   21]
 [  40  281]]

True positives 281
True negatives 3238
False positives 21
False negatives 40


## Presenting

The results will be presented with a poster. As a first draft, the poster will include the following:

* Problem statement: similar to what's included in deliverable 2
* Background: motivation for choosing the project, what are pulsars and why does the problem exist
* The data used: description of the data, similar to what's in deliverable 1 and 2
* Methodology: similar to what's in deliverable 1, but also including the process of selecting the final model as descriped in this deliverable
* Results: the f1-scores and confusion matrix from this deliverable
* Discussion: compare the results to baseline performance